<a href="https://colab.research.google.com/github/Shifoue/Segmentation/blob/main/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Shifoue/Portfolio.git

fatal: destination path 'Portfolio' already exists and is not an empty directory.


In [2]:
!cd Portfolio/Eye_Segmentation_Project/

In [3]:
!pip install utils
!pip install torchmetrics

In [4]:
!unzip Dataset_Faces_training.zip
!unzip Dataset_Faces_validation.zip

!unzip Dataset_Faces_Mask_training.zip
!unzip Dataset_Faces_Mask_validation.zip

unzip:  cannot find or open Dataset_Faces_training.zip, Dataset_Faces_training.zip.zip or Dataset_Faces_training.zip.ZIP.
unzip:  cannot find or open Dataset_Faces_validation.zip, Dataset_Faces_validation.zip.zip or Dataset_Faces_validation.zip.ZIP.
unzip:  cannot find or open Dataset_Faces_Mask_training.zip, Dataset_Faces_Mask_training.zip.zip or Dataset_Faces_Mask_training.zip.ZIP.
unzip:  cannot find or open Dataset_Faces_Mask_validation.zip, Dataset_Faces_Mask_validation.zip.zip or Dataset_Faces_Mask_validation.zip.ZIP.


In [5]:
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.depht = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, X):
        return self.depht(X)

class myUNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
      super(myUNET, self).__init__()
      self.encode = nn.ModuleList()
      self.decode = nn.ModuleList()
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #divide size per 2

      # Encode
      for feature in features:
          self.encode.append(DoubleConv(in_channels, feature))
          in_channels = feature

      # Decode
      for feature in reversed(features):
          self.decode.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
          self.decode.append(DoubleConv(feature*2, feature))

      self.bottleneck = DoubleConv(features[-1], features[-1]*2)
      self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, X):
        skip_connections = []

        for depht in self.encode:
            X = depht(X)
            skip_connections.append(X)
            X = self.pool(X)

        X = self.bottleneck(X)
        skip_connections = skip_connections[::-1]

        for i in range(0, len(self.decode), 2):
            X = self.decode[i](X)
            #skip_connection = skip_connections[-1 - i//2]
            skip_connection = skip_connections[i//2]

            if X.shape != skip_connection.shape:
                X = torch.nn.functional.interpolate(X, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, X), dim=1)
            X = self.decode[i+1](concat_skip)

        return self.final_conv(X)

def test():
    x = torch.randn((3, 1, 160, 160))
    m = myUNET(in_channels=1, out_channels=1)
    pred = m(x)

    assert pred.shape == x.shape

test()

In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

In [7]:
class FaceDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.masks[index])
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L") #, dtype=np.float32)

        #mask[mask == 255.0] = 1.0
        #mask = mask/255.0

        if self.transform is not None:

            #augmentations = self.transform(image=image, mask=mask)
            image = self.transform(image) #augmentations["image"]
            mask = self.transform(mask) #augmentations["mask"]
            mask = (mask > 0.5).float()

        return image, mask

In [8]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [9]:
#Hyperparamters

LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 30
NUM_WORKERS = 2
IMAGE_HEIGHT = 218
IMAGE_WIDTH = 178
PIN_MEMORY = True
LOAD_MODEL = True
IMG_DIR_TRAIN = "/content/Portfolio/Eye_Segmentation_Project/Dataset_Faces_training"
IMG_DIR_VAL = "/content/Portfolio/Eye_Segmentation_Project/Dataset_Faces_validation"
MASK_DIR_TRAIN = "/content/Portfolio/Eye_Segmentation_Project/Dataset_Faces_Mask_training"
MASK_DIR_VAL = "/content/Portfolio/Eye_Segmentation_Project/Dataset_Faces_Mask_validation"
SAVE_DIR = "Saved_Images"

In [10]:
import cv2

In [11]:
from torch.utils.data import DataLoader

def save_checkpoint(state, filename="checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_mask_dir,
    val_dir,
    val_mask_dir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True
):
    train_data = FaceDataset(image_dir=train_dir, mask_dir=train_mask_dir, transform = train_transform)
    train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True)

    val_data = FaceDataset(image_dir=val_dir, mask_dir=val_mask_dir, transform = val_transform)
    val_loader = DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False)

    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0 #did i predict correctly
    num_pixels = 0 #how many pixel are correct Area accuracy

    #In order to force the algorithm on finding a good solution instead of only putting black pixel to get a good accuracy due to the small size of the target
    dice_score = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8) #calculate the intersection between ground truth and prediction

    print(f"Got {num_correct}/{num_pixels} with accuracy {num_correct/num_pixels*100}:.2f")
    print(f"Dice score {dice_score/len(loader)}")

    model.train()

def save_predictions_as_imgs(loader, model, folder = SAVE_DIR, device="cuda"):
    if not os.path.exists(folder):
      os.mkdir(folder)

    model.eval()

    for i , (x, y) in enumerate(loader):
        x = x.to(device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            #print(y.shape)
            #print(preds.shape)
            #y = (y > 0.5).float()

            # preds_path = os.path.join(folder, f"pred_{i}.jpg")
            # truth_path = os.path.join(folder, f"truth_{i}.jpg")
            # print(preds_path)
            # print(truth_path)


            #cv2.imwrite(preds_path, preds.cpu().detach().numpy())
            #cv2.imwrite(truth_path, y.cpu().detach().numpy())
            torchvision.utils.save_image(preds, f"{folder}/pred_{i}.jpg")
            torchvision.utils.save_image(y, f"{folder}/truth_{i}.jpg")

In [12]:
def train(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for i_batch, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        with torch.cuda.amp.autocast():
            preds = model(data)
            loss = loss_fn(preds, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss=loss.item())

In [13]:
import torch.nn.functional as F

In [14]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1. - dice

In [15]:
train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
        #A.Rotate(limit=35, p=1.0),
        #A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.1),
        #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        #transforms.Grayscale(),
    ]
)

validation_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
    ]
)

UNET = myUNET(in_channels=3, out_channels=1).to(DEVICE) #for multiple classes change out channels to number of classes
#loss_fn = nn.BCEWithLogitsLoss() # for multiple classes use cross entropy

loss_fn = DiceLoss() #Needed to force the NN to chose another strategy than putting every pixel to white
optimizer = optim.Adam(UNET.parameters(), lr=LEARNING_RATE)

train_loader, val_loader = get_loaders(
    IMG_DIR_TRAIN,
    MASK_DIR_TRAIN,
    IMG_DIR_VAL,
    MASK_DIR_VAL,
    BATCH_SIZE,
    train_transform,
    validation_transform,
    NUM_WORKERS,
    PIN_MEMORY
)

scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):
    #print(train_loader)
    train(train_loader, UNET, optimizer, loss_fn, scaler)

    checkpoint =  {
        "state_dict": UNET.state_dict(),
        "optimizer": optimizer.state_dict()
    }

    save_checkpoint(checkpoint)

    check_accuracy(val_loader, UNET, device=DEVICE)

    save_predictions_as_imgs(val_loader, UNET, folder="/content/Saved_Images", device=DEVICE)

  0%|          | 0/50 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 50/50 [00:14<00:00,  3.36it/s, loss=0.981]


=> Saving checkpoint
Got 7601927/7760800 with accuracy 97.95287322998047:.2f
Dice score 0.21751512587070465


100%|██████████| 50/50 [00:11<00:00,  4.33it/s, loss=0.243]


=> Saving checkpoint
Got 7747275/7760800 with accuracy 99.82572174072266:.2f
Dice score 0.7050676345825195


100%|██████████| 50/50 [00:11<00:00,  4.27it/s, loss=0.274]


...

=> Saving checkpoint
Got 7753919/7760800 with accuracy 99.91133117675781:.2f
Dice score 0.8448656797409058


100%|██████████| 50/50 [00:11<00:00,  4.33it/s, loss=0.133]


=> Saving checkpoint
Got 7753334/7760800 with accuracy 99.90379333496094:.2f
Dice score 0.845048189163208


100%|██████████| 50/50 [00:11<00:00,  4.36it/s, loss=0.0835]


=> Saving checkpoint
Got 7753487/7760800 with accuracy 99.90576171875:.2f
Dice score 0.8432580828666687


In [16]:
import matplotlib.pyplot as plt

truth_image = Image.open('/content/Saved_Images/truth_0.jpg')
pred_image = Image.open('/content/Saved_Images/pred_0.jpg')


figure_size = 15
plt.figure(figsize=(figure_size,figure_size))
plt.subplot(1,2,1),plt.imshow(truth_image)
plt.title('Truth Image'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(pred_image)
plt.title('Prediction Image'), plt.xticks([]), plt.yticks([])
plt.show()